## Agents and Tools

### Agents: Agents enable LLMs to work with tools that perform various tasks
To use agents, we require three things:
- A base LLM,
- A tool that we will be interacting with,
- An agent to control the interaction.

### In the following example, we will use the Math tool to perform some simple math operations

In [1]:
from dotenv import load_dotenv
from langchain_openai import OpenAI
load_dotenv()

True

In [3]:
llm = OpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo-instruct"
)

In [4]:
from langchain.chains import LLMMathChain # Chain that interprets a prompt and executes python code to do math.
from langchain.agents import Tool

In [6]:
# initialize the math tool

llm_math = LLMMathChain(llm=llm)

math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)

# when giving tools to LLM, we must pass as list of tools
tools = [math_tool]  # is a list of possible Tools that your agent can use

C:\Users\rakes\anaconda3\lib\site-packages\langchain\chains\llm_math\base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [7]:
print(tools[0].name, "---", tools[0].description)

Calculator --- Useful for when you need to answer questions about math.


## Now create an Agent
#### Zero-shot ReAct
The Zero-shot ReAct Agent is a language generation model that can create realistic contexts even without being trained on specific data. It can be used for various tasks such as generating creative text formats, language translation, and generating different types of creative content.

In [8]:
from langchain.agents import initialize_agent

zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

C:\Users\rakes\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
zero_shot_agent("What is root over 25")

C:\Users\rakes\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use a calculator to find the square root of 25
Action: Calculator
Action Input: √25
Observation: Answer: 5.0
Thought: I now know the final answer
Final Answer: 5.0

> Finished chain.


{'input': 'What is root over 25', 'output': '5.0'}

In [10]:
problem = """
        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen 
        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. 
        What is the total area of your house?
        """
zero_shot_agent(problem)



> Entering new AgentExecutor chain...
 I need to calculate the area of each room and then add them together to get the total area of the house.
Action: Calculator
Action Input: 5 * 5
Observation: Answer: 25
Thought: This is the area of one bedroom.
Action: Calculator
Action Input: 7 * 6
Observation: Answer: 42
Thought: This is the area of the drawing cum open kitchen.
Action: Calculator
Action Input: 3 * 2
Observation: Answer: 6
Thought: This is the area of the balcony.
Action: Calculator
Action Input: 25 + 25 + 42 + 6
Observation: Answer: 98
Thought: This is the total area of the house.
Final Answer: The total area of the house is 98 square metres.

> Finished chain.


{'input': '\n        You are building a house. There are two bedrooms of 5 metres by 5 metres each and drawing cum open kitchen \n        is 7 metres by 6 metres and balcony of 3 metres by 2 metres. \n        What is the total area of your house?\n        ',
 'output': 'The total area of the house is 98 square metres.'}

## Using mutiple Tools
In the following example, we will be using the Duck Duck Go search API as a Tool combined with the Math Tool
We will then give the Agent a problem that initiates a search first followed by a Calculation.

In [28]:
# !pip install -U duckduckgo-search

  Using cached duckduckgo_search-6.1.0-py3-none-any.whl (23 kB)
  Using cached pyreqwest_impersonate-0.4.5-cp38-abi3-win_amd64.whl (2.6 MB)


In [29]:
from langchain_community.tools import DuckDuckGoSearchRun
# search = DuckDuckGoSearchRun()

from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()
search.run("Obama")

'[snippet: Barack Obama—with his wife, Michelle—being sworn in as the 44th president of the United States, January 20, 2009. Key events in the life of Barack Obama. Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) is the 44th president of the United States (2009-17) and the first African American to hold the office., title: Barack Obama | Biography, Parents, Education, Presidency, Books ..., link: https://www.britannica.com/biography/Barack-Obama], [snippet: 1:12. Former President Barack Obama, in a CNN interview that aired Thursday night, said he does not believe President Joe Biden will face a serious primary challenge during his 2024 reelection ..., title: Five takeaways from Barack Obama\'s CNN interview on Biden, Trump, 2024, link: https://www.usatoday.com/story/news/politics/2023/06/23/five-takeaways-from-barack-obama-cnn-interview/70349112007/], [snippet: In private meeting, Obama told Biden he would do whatever it takes to help in 2024. The former president made the c

In [34]:
# Define the Tools
# import duckduckgo
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search_tool = Tool.from_function(
    func=search.run,
    name="Search",
    description="useful for when you need to search the internet for information"
)

llm_math_chain = LLMMathChain(llm=llm, verbose=True)

math_tool = Tool.from_function(
    func=llm_math_chain.run,
    name="Calculator",
    description="Useful for when you are asked to perform math calculations"
)

C:\Users\rakes\anaconda3\lib\site-packages\langchain\chains\llm_math\base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [35]:
# Define the Agent

tools = [search_tool, math_tool]

agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [36]:
# Run the Agent with the problem

agent.run("""Get Microsoft Stock Price taken from Google Finance and display in both USD and INR values""")



> Entering new AgentExecutor chain...
 I should use the search tool to find the current Microsoft stock price on Google Finance.
Action: Search
Action Input: "Microsoft stock price Google Finance"
Observation: Shares of Microsoft rose Tuesday, on track for their first record close in two months. The software giant's shares, up about 10% this month, were recently up more than 1% at over $430 apiece. They ... Discover real-time Microsoft Corporation Common Stock (MSFT) stock prices, quotes, historical data, news, and Insights for informed trading and investment decisions. Stay ahead with Nasdaq. Stock analysis for Microsoft Corp (MSFT:NASDAQ GS) including stock price, stock chart, company news, key statistics, fundamentals and company profile. Get a real-time Microsoft Corporation (MSFT) stock price quote with breaking news, financials, statistics, charts and more. ... Expect a fusillade of news showing that Microsoft has as much artificial intelligence as Google, as well as Apple, its

'The current Microsoft stock price is $430 USD and 35,789.20 INR.'

## Create a custom tool

In [37]:
from langchain.tools import BaseTool
from langchain_core.output_parsers import JsonOutputParser
from langchain.tools import tool

@tool("JSON_Response", return_direct=True)
def StructuredResponseTool(question: str):
    """
    use this tool to send a prompt and get a JSON returned 
    with three fields - Topic, Question_Details and Detailed_Response
    """
    json_prompt = PromptTemplate.from_template(
    """Return a JSON object with an `answer` key that answers the following question: {question}. 
        The JSON object will have three fields - Topic, Question_Details and Detailed_Response"""
    )

    model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)

    json_parser = JsonOutputParser()
    json_chain = json_prompt | model | json_parser
    x = json_chain.invoke({"question": question})
    return x

In [38]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [41]:
from langchain.agents import initialize_agent

tools = [StructuredResponseTool]

# initialize agent with tools
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=10
)

In [42]:
zero_shot_agent.run(input="Which is the top football team in the world and how, the response should have three fields in the JSON - topic of the question, the question and the detailed response")



> Entering new AgentExecutor chain...
 I should use the JSON_Response tool to get the answer
Action: JSON_Response
Action Input: "Which is the top football team in the world and how"

OutputParserException: Invalid json output: Topic: Football
Question_Details: Which is the top football team in the world and how?
Detailed_Response: The top football team in the world is currently considered to be Liverpool FC. This is based on their recent success in the Premier League, where they won the title in the 2019-2020 season with a record-breaking 99 points. They also won the UEFA Champions League in 2019 and the FIFA Club World Cup in 2019, solidifying their position as the top team in the world. Their success can be attributed to their strong team chemistry, talented players, and exceptional coaching by Jurgen Klopp. Liverpool FC's high-intensity, attacking style of play has also been a key factor in their dominance in recent years. Overall, their consistent performance and impressive trophy haul make them the top football team in the world.